In [1]:
import pickle
from transformers import AutoTokenizer
import numpy as np
import shap

from datasets import load_dataset

import numpy as np
from src.run_shap import load_shap_vals
import pickle
from tqdm import tqdm
from src.utils import token_segments
from src.utils import legacy_get_dataset_info
from transformers import AutoTokenizer
import shap
from src.utils import token_segments, text_ft_index_ends, format_text_pred, ConfigLoader

# %matplotlib inline
import matplotlib.pyplot as plt

# plt.style.use('seaborn-whitegrid')
import numpy as np
import pandas as pd
from src.run_shap import load_shap_vals
from tqdm import tqdm

# import re
# import seaborn as sns
# from scipy import stats
from src.utils import (
    ConfigLoader,
)
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

/home/james/CodingProjects/TextNTabularExplanations/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
all_dfs = {}
for ds_name in [
    "fake",
    "jigsaw",
    "kick",
    "wine",
    "imdb_genre",
    "prod_sent",
    "salary",
    "channel",
    "airbnb",
]:
    df1 = pd.DataFrame(columns=["config", "col", "val", "total"])
    for text_model_base, text_model_code in [
        ("distilbert-base-uncased", "disbert"),
        ("distilroberta-base", "drob"),
        ("microsoft/deberta-v3-small", "deberta"),
        ("bert-base-uncased", "bert"),
    ]:
        tab_scale_factor = 1
        # Shap args
        # args = ConfigLoader(config_type, "configs/shap_configs.yaml")
        # Dataset info
        di = ConfigLoader(ds_name, "../configs/dataset_configs.yaml")
        tokenizer = AutoTokenizer.from_pretrained(text_model_base)

        pre = f"_sf{tab_scale_factor}" if tab_scale_factor != 2 else ""
        col_name_filepath = f"../models/shap_vals_{text_model_code}{pre}/{ds_name}/col_names_shap_vals_all_text_baseline.pkl"
        colon_filepath = f"../models/shap_vals_{text_model_code}{pre}/{ds_name}/colon_shap_vals_all_text_baseline.pkl"
        fts_filepath = f"../models/shap_vals_{text_model_code}{pre}/{ds_name}/summed_shap_vals_all_text_baseline.pkl"

        tab_pre = f"_sf{tab_scale_factor}" if tab_scale_factor != 2 else ""
        print(
            f"""
            #################
            {ds_name}
            #################
            """
        )
        with open(col_name_filepath, "rb") as f:
            grouped_col_name_shap_vals = pickle.load(f)
        with open(colon_filepath, "rb") as f:
            grouped_colon_shap_vals = pickle.load(f)
        with open(fts_filepath, "rb") as f:
            grouped_shap_vals = pickle.load(f)
        val_only = (
            grouped_shap_vals - grouped_col_name_shap_vals - grouped_colon_shap_vals
        )
        col_plus_colon = grouped_col_name_shap_vals + grouped_colon_shap_vals

        val_fi = np.mean(np.sum(np.abs(val_only), axis=0), axis=0)
        tmp_fi = np.mean(np.sum(np.abs(col_plus_colon), axis=0), axis=0)

        tab_val_fi = val_fi[: len(di.tab_cols)]
        tab_tmp_fi = tmp_fi[: len(di.tab_cols)]
        text_val_fi = val_fi[len(di.tab_cols) :]
        text_tmp_fi = tmp_fi[len(di.tab_cols) :]

        items = {
            "text": np.sum(text_val_fi),
            "tab": np.sum(tab_val_fi),
            "text template": np.sum(text_tmp_fi),
            "tab template": np.sum(tab_tmp_fi),
        }
        for i, (k, v) in enumerate(items.items()):
            df1 = pd.concat(
                [
                    df1,
                    pd.DataFrame(
                        [
                            {
                                "config": text_model_code,
                                "col": k,
                                "val": v,
                                "total": 1,
                                "col_index": i,
                            }
                        ]
                    ),
                ],
                ignore_index=True,
            )
    all_dfs[ds_name] = df1
# df1["showlegend"] = df1.groupby("col")["config"].cumcount() == 0
# colors = px.colors.qualitative.Plotly

# fig = go.Figure()
# for index, row in df1.iterrows():
#     fig.add_trace(
#         go.Bar(
#             y=[row["config"]],
#             # y=[1],
#             x=[row["val"]],
#             # customdata=[row["gdpPercap"]],
#             # hovertemplate="%{x:2f}, GDP per capita: %{customdata:2f}",
#             marker=dict(color=colors[int(row["col_index"])]),
#             name=row["col"],
#             legendgroup=row["col"],
#             showlegend=row["showlegend"],
#             orientation="h",
#         )
#     )

# fig.update_layout(barmode="stack", hovermode="y unified")

# # show figure
# fig.show()


{'config': 'fake', 'tab_cols': ['required_experience', 'required_education'], 'categorical_cols': ['required_experience', 'required_education'], 'text_cols': ['title', 'description', 'salary_range'], 'label_names': ['0', '1'], 'label_col': 'fraudulent', 'num_labels': 2, 'prob_type': 'single_label_classification', 'wandb_proj_name': 'Fake Job Postings', 'all_text_dataset': 'james-burton/fake_job_postings2_all_text', 'ordinal_dataset': 'james-burton/fake_job_postings2_ordinal', 'tnt_reorder_cols': {'bert': ['description', 'title', 'required_experience', 'required_education', 'salary_range'], 'disbert': ['description', 'title', 'required_education', 'required_experience', 'salary_range'], 'drob': ['description', 'title', 'required_education', 'required_experience', 'salary_range'], 'deberta': ['description', 'title', 'required_education', 'required_experience', 'salary_range']}, 'base_reorder_cols': {'bert': ['description', 'title', 'required_education', 'required_experience', 'salary_ra

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



            #################
            fake
            #################
            

{'config': 'fake', 'tab_cols': ['required_experience', 'required_education'], 'categorical_cols': ['required_experience', 'required_education'], 'text_cols': ['title', 'description', 'salary_range'], 'label_names': ['0', '1'], 'label_col': 'fraudulent', 'num_labels': 2, 'prob_type': 'single_label_classification', 'wandb_proj_name': 'Fake Job Postings', 'all_text_dataset': 'james-burton/fake_job_postings2_all_text', 'ordinal_dataset': 'james-burton/fake_job_postings2_ordinal', 'tnt_reorder_cols': {'bert': ['description', 'title', 'required_experience', 'required_education', 'salary_range'], 'disbert': ['description', 'title', 'required_education', 'required_experience', 'salary_range'], 'drob': ['description', 'title', 'required_education', 'required_experience', 'salary_range'], 'deberta': ['description', 'title', 'required_education', 'required_experience', 'salary_range']}, 'base_reorder_cols'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



            #################
            jigsaw
            #################
            

{'config': 'jigsaw', 'tab_cols': ['asian', 'atheist', 'bisexual', 'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu', 'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability', 'jewish', 'latino', 'male', 'muslim', 'other_disability', 'other_gender', 'other_race_or_ethnicity', 'other_religion', 'other_sexual_orientation', 'physical_disability', 'psychiatric_or_mental_illness', 'transgender', 'white', 'funny', 'wow', 'sad', 'likes', 'disagree'], 'categorical_cols': [], 'text_cols': ['comment_text'], 'label_names': ['False', 'True'], 'label_col': 'target', 'num_labels': 2, 'prob_type': 'single_label_classification', 'wandb_proj_name': 'Jigsaw', 'all_text_dataset': 'james-burton/jigsaw_unintended_bias100K_all_text', 'ordinal_dataset': 'james-burton/jigsaw_unintended_bias100K_ordinal', 'tnt_reorder_cols': {'bert': ['comment_text', 'atheist', 'other_gender', 'buddhist', 'h

...


            #################
            salary
            #################
            

{'config': 'salary', 'tab_cols': ['job_type'], 'categorical_cols': ['job_type'], 'text_cols': ['experience', 'job_description', 'job_desig', 'key_skills', 'location'], 'label_names': ['6to10', '10to15', '0to3', '15to25', '3to6', '25to50'], 'label_col': 'salary', 'num_labels': 6, 'prob_type': 'single_label_classification', 'wandb_proj_name': 'Salary', 'all_text_dataset': 'james-burton/data_scientist_salary_all_text', 'ordinal_dataset': 'james-burton/data_scientist_salary_ordinal', 'tnt_reorder_cols': {'bert': ['experience', 'job_desig', 'key_skills', 'job_description', 'location', 'job_type'], 'disbert': ['experience', 'job_description', 'location', 'job_desig', 'key_skills', 'job_type'], 'drob': ['experience', 'job_desig', 'key_skills', 'job_description', 'job_type', 'location'], 'deberta': ['experience', 'key_skills', 'job_desig', 'job_description', 'location', 'job_type']}, 'base_reorder_col

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



            #################
            channel
            #################
            

{'config': 'channel', 'tab_cols': [' n_tokens_content', ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens', ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos', ' average_token_length', ' num_keywords', ' global_subjectivity', ' global_sentiment_polarity', ' global_rate_positive_words', ' global_rate_negative_words', ' rate_positive_words', ' rate_negative_words'], 'categorical_cols': [], 'text_cols': ['article_title'], 'label_names': [' data_channel_is_tech', ' data_channel_is_entertainment', ' data_channel_is_lifestyle', ' data_channel_is_bus', ' data_channel_is_world', ' data_channel_is_socmed'], 'label_col': 'channel', 'num_labels': 6, 'prob_type': 'single_label_classification', 'wandb_proj_name': 'News Channel', 'all_text_dataset': 'james-burton/news_channel_all_text', 'ordinal_dataset': 'james-burton/news_channel_ordinal', 'tnt_reorder_cols': {'bert': ['article

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



            #################
            airbnb
            #################
            

{'config': 'airbnb', 'tab_cols': ['accommodates', 'availability_30', 'availability_365', 'availability_60', 'availability_90', 'bathrooms', 'bed_type', 'bedrooms', 'beds', 'cancellation_policy', 'city', 'cleaning_fee', 'extra_people', 'guests_included', 'host_identity_verified', 'host_is_superhost', 'host_response_time', 'instant_bookable', 'is_location_exact', 'latitude', 'license', 'longitude', 'maximum_nights', 'minimum_nights', 'number_of_reviews', 'review_scores_accuracy', 'review_scores_checkin', 'review_scores_cleanliness', 'review_scores_communication', 'review_scores_location', 'review_scores_rating', 'review_scores_value', 'reviews_per_month', 'room_type', 'security_deposit'], 'categorical_cols': ['bed_type', 'cancellation_policy', 'city', 'host_identity_verified', 'host_is_superhost', 'host_response_time', 'instant_bookable', 'is_location_exact', 'license', 'room_type'], 'text_cols'

In [11]:
import kaleido

for ds_name in [
    "fake",
    "jigsaw",
    "kick",
    "wine",
    "imdb_genre",
    "prod_sent",
    "salary",
    "channel",
    "airbnb",
]:
    print(ds_name)
    df1 = all_dfs[ds_name]
    df1["showlegend"] = df1.groupby("col")["config"].cumcount() == 0
    colors = px.colors.qualitative.Plotly[2:]

    fig = go.Figure()
    fig.update_layout(
        barmode="stack",
        hovermode="y unified",
        height=200,  # Set your desired height
        width=800,  # Set your desired width
        legend=dict(
            orientation="h",  # Set orientation to horizontal
            yanchor="top",
            y=1.25,
            xanchor="left",
            x=0,
        ),
        margin=dict(l=0, r=0, t=0, b=0),
    )
    for index, row in df1.iterrows():
        fig.add_trace(
            go.Bar(
                y=[row["config"]],
                # y=[1],
                x=[row["val"]],
                # customdata=[row["gdpPercap"]],
                # hovertemplate="%{x:2f}, GDP per capita: %{customdata:2f}",
                marker=dict(color=colors[int(row["col_index"])]),
                name=row["col"],
                legendgroup=row["col"],
                showlegend=row["showlegend"],
                orientation="h",
            )
        )

    fig.update_layout(barmode="stack", hovermode="y unified")
    fig.show()

    # save to file
    fig.write_image(f"images/template_proportion/{ds_name}.pdf")

fake


jigsaw


kick


wine


imdb_genre


prod_sent


salary


channel


airbnb


In [27]:
colors[row["col_index"]]

TypeError: list indices must be integers or slices, not float

In [ ]:
df1

,config,col,val,total,col_index,showlegend
0,disbert,text_fi,0.342546,1,0.0,True
1,disbert,tab_fi,0.044077,1,1.0,True
2,disbert,text_tmp_fi,0.028449,1,2.0,True
3,disbert,tab_tmp_fi,0.076761,1,3.0,True
4,drob,text_fi,0.480954,1,0.0,False
5,drob,tab_fi,0.048746,1,1.0,False
6,drob,text_tmp_fi,0.014054,1,2.0,False
7,drob,tab_tmp_fi,0.135704,1,3.0,False
8,deberta,text_fi,0.548073,1,0.0,False
9,deberta,tab_fi,0.018577,1,1.0,False


## Proportion charts

In [ ]:
configs = [
    "vet_19d_stack",
    "vet_59d_stack",
    "vet_19d_ensemble_25",
    "vet_59d_ensemble_25",
    "vet_19d_ensemble_50",
    "vet_59d_ensemble_50",
    "vet_19d_ensemble_75",
    "vet_59d_ensemble_75",
    "vet_10d_all_text",
    # "vet_10c_2_all_text",
    "vet_50d_all_text",
]

c2c_dict = {
    "vet_19d_ensemble_25": "A: BERT WE w=.25",
    "vet_59d_ensemble_25": "B: PetBERT WE w=.25",
    "vet_19d_ensemble_50": "C: BERT WE w=.50",
    "vet_59d_ensemble_50": "D: PetBERT WE w=.50",
    "vet_19d_ensemble_75": "E: BERT WE w=.75",
    "vet_59d_ensemble_75": "F: PetBERT WE w=.75",
    "vet_19d_stack": "G: BERT Stack",
    "vet_59d_stack": "H: PetBERT Stack",
    "vet_10d_all_text": "I: BERT All Text",
    "vet_50d_all_text": "J: PetBERT All Text",
}


val_dict = {}
for config in configs:
    with open(f"../models/shap_vals/summed_{config}.pkl", "rb") as f:
        grouped_shap_vals = pickle.load(f)
    val_dict[config] = (
        abs(grouped_shap_vals).sum(axis=0).mean(axis=0)
        / abs(grouped_shap_vals).sum(axis=0).mean(axis=0).sum()
    )
df1 = pd.DataFrame(columns=["config", "col", "val", "total"])
for idx, config in enumerate(configs):
    vals = val_dict[config]
    args = ConfigLoader(
        config, "../configs/shap_configs.yaml", "../configs/dataset_default.yaml"
    )
    cols = args.categorical_cols + args.numerical_cols + args.text_cols
    for i, col in enumerate(cols):
        df1 = pd.concat(
            [
                df1,
                pd.DataFrame(
                    [
                        {
                            "config": c2c_dict[config],
                            "col": col,
                            "val": vals[i],
                            "total": 1,
                        }
                    ]
                ),
            ],
            ignore_index=True,
        )
# rename rows in col
# record -> (Text ft) clinical narrative
# Dental - > dental
# age_at_consult -> age at consult
# region -> (Text ft) region
# breed -> (Text ft) breed
# Neoplasms -> neoplasms
# Nerous -> nervous

df1.loc[df1["col"] == "record", "col"] = "(Text) clin. narr."
df1.loc[df1["col"] == "Dental", "col"] = "dental"
df1.loc[df1["col"] == "age_at_consult", "col"] = "age at consult"
df1.loc[df1["col"] == "region", "col"] = "(Text) region"
df1.loc[df1["col"] == "breed", "col"] = "(Text) breed"
df1.loc[df1["col"] == "Neoplasms", "col"] = "neoplasms"
df1.loc[df1["col"] == "Nervous", "col"] = "nervous"

# country_index = col_index
# year = config
# country = col
# pop = val
# cols_to_use = set(df1[df1["val"] > 0.05]['col'].values)
cols_to_use = (
    df1.groupby("col")["val"]
    .sum()
    .reset_index()
    .sort_values("val", ascending=False)["col"]
    .values[:6]
)
# if col not in cols_to_use: set col = 'other'
df1.loc[~df1["col"].isin(cols_to_use), "col"] = "other"
# cols_to_use.add('other')
# cols_to_use = list(cols_to_use)
cols_to_use = list(cols_to_use) + ["other"]
# df1 = df1.sort_values(["val"], ignore_index=True, ascending=False)
# df1 = df1.sort_values(["config"], key=lambda x: x.map(dict(zip(configs, range(len(configs))))), ignore_index=False, ascending=True)
# make col index based on cols_to_use
df1["col_index"] = df1["col"].apply(lambda x: cols_to_use.index(x))
# if col == 'other': set col_index = max(col_index) + 1
df1.loc[df1["col"] == "other", "col_index"] = df1["col_index"].max() + 1
# df1 = df1.sort_values(["config"], ignore_index=True, ascending=True)
# df1 = df1.sort_values(["col_index"], ignore_index=True, ascending=True)


# custom_order = dict(zip(range(len(cols_to_use)),configs ))
# df1 = df1.sort_values(by=["col_index", "config"], ignore_index=True, ascending=[True, True], key=lambda x: x.map(custom_order))


# # custom_order = dict(zip(configs, range(len(configs)) ))
# # df1 = df1.sort_values(by=["config", "col_index" ], ignore_index=False, ascending=[True, True], key=lambda x: x.map(custom_order))
# # df1 =
# # # df1 = df1.sort_values(by=["col_index", "config"], ignore_index=True, ascending=[True, True], key=lambda x: x.map(custom_order))

df1 = df1.sort_values(
    ["col_index", "config"], ignore_index=True, ascending=[True, False]
)
#
# colors = px.colors.qualitative.Set3[:len(cols)]


df1["showlegend"] = df1.groupby("col")["config"].cumcount() == 0
# + px.colors.qualitative.Pastel + px.colors.qualitative.Bold + px.colors.qualitative.Light24
colors = px.colors.qualitative.Plotly

fig = go.Figure()
for index, row in df1.iterrows():
    fig.add_trace(
        go.Bar(
            y=[row["config"]],
            # y=[1],
            x=[row["val"]],
            # customdata=[row["gdpPercap"]],
            # hovertemplate="%{x:2f}, GDP per capita: %{customdata:2f}",
            marker=dict(color=colors[row["col_index"]]),
            name=row["col"],
            legendgroup=row["col"],
            showlegend=row["showlegend"],
            orientation="h",
        )
    )

fig.update_layout(barmode="stack", hovermode="y unified")